# Example on how to create an adaptive fast 2D-scan, using the Basel Precision Instruments LNHR DAC II QCoDeS driver

Copyright (c) Basel Precision Instruments GmbH (2025)

...............................................................................................................

## Disclaimer

The driver has always supported every command that you can use to control the DAC. However, the way those commands are implemented is not in line with how QCoDeS normally (should) work(s). This means, that programming complicated functions with this driver often yields code that cannot interface with other QCoDeS-code nicely. One such complicated function would be a fast adaptive 2D scan. And since the possibility to do this with the QCoDeS driver exists, we think it is beneficial to show how it works, in disregard on how nicely it fits the concept behind QCoDeS.

This notebook is basically a translation from the file `LNHRDACII-tools/fast_2D_scan_example.ipynb` into code that runs with the QCoDeS driver instead the Non-QCoDeS Telnet driver.

## 0 - introduction

The LNHR DAC II is by default equipped with all the necessary functions to perform a fast adaptive 2D-scan. In a typical fast adaptive 2D-scan scenario, two outputs of the LNHR DAC II are used to create the x-axis and y-axis signals. Optionally a third output can be used to create a trigger signal. Alternitavely the LNHR DAC II can be triggered by the Data Acquisition, or no trigger can be used at all. The picture below shows a diagram of such a typical 2D-scan setup.

<img src="./graphics/typical-measurement-setup.png" width="1308" height="215">

## 1 - imports and setup
For this example the Basel Precision Instruments LNHR DAC II qcodes-driver is used (available on Github).

In [1]:
# Baspi Dac driver
from baspi_lnhrdac2 import SP1060

#create a dac instance:
dac = SP1060('LNHR_dac', 'TCPIP0::192.168.0.5::23::SOCKET')  

Connected to: Basel Precision Instruments LNHR DAC II (SP1060) (serial: SN 10600000011, firmware: Revision 3.4.9u) in 0.05s
Current DAC output (Channel 1 ... Channel 24): 
(1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0)


## 2 - define parameters for the 2D-scan
To simplify the setup process of the 2D-scan, a few parameters are defined here, from which the actual DAC parameters are derived from:

- **x_steps**: number of steps on the slower x-axis (int)
- **x_start_voltage**: voltage at which the slower x-axis starts a sweep (float)
- **x_stop-voltage**: voltage at which the slower x-axis stops a sweep (float)
- **y_steps**: number of steps on the faster y-axis (int)
- **y_start_voltage**: voltage at which the faster y-axis starts a sweep (float)
- **y_stop-voltage**: voltage at which the faster y-axis stops a sweep (float)
- **acquisition_delay**: time the data acquisition needs to measure the signal in ms (milli-seconds) or "duration of a point" (float)
- **adaptive_shift**: voltage shift after each fast sweep of the y-axis

In [2]:
x_steps = 10
x_start_voltage = 0.0
x_stop_voltage = 1.0
y_steps = 10
y_start_voltage = 0.0
y_stop_voltage = 1.0
acquisition_delay = 10
adaptive_shift = 0.1

## 3 - choose DAC channels for x- and y-axis
The output for the x- and y-axis can be selected freely, as long as both outputs are on the same DAC-Board (channels 1 - 12 or 13 - 24). For this example the AWG-A and RAMP-A generators are used.


In [3]:
#define the DAC channels which will be used as outputs
output_x = 1
output_y = 2

#check availability of the AWG-A and RAMP-A, set auxiliary variables
if dac.read_AWGChannelAvailable(awg="A") == "1" and dac.read_rampChannelAvailable(ramp="A") == "1":
    pass
else:
    raise Exception("AWG resources not available")

awg_output = "a"
memory = 0
board = "ab"

## 4 - setup x-axis
The LNHR DAC II must be configured to automatically update the slower x-axis signal using the before chosen ramp generator.

<img src="./graphics/block-diagram-xaxis.png" width="1308" height="215">

In [4]:
# calculate the internally used parameter
ramp_time = 0.005 * (x_steps + 1)

dac.set_ramp_channel(ramp=f"{awg_output}",          channel=f"{output_x}")      #ramp selected DAC-channel      
dac.set_ramp_starting_voltage(ramp=f"{awg_output}", voltage= x_start_voltage)   #ramp start voltage
dac.set_ramp_peak_voltage(ramp=f"{awg_output}",     voltage= x_stop_voltage)    #ramp stop voltage
dac.set_ramp_duration(ramp=f"{awg_output}",         time= ramp_time)            #ramp time
dac.set_ramp_shape(ramp=f"{awg_output}",            shape= 0)                   #ramp shape
dac.set_ramp_cycles(ramp=f"{awg_output}",           cycles= 1)                  #ramp cycles set
dac.select_ramp_step(ramp=f"{awg_output}",          mode= 1)                    #ramp/step selection


'0'

# 5 - setup y-axis
The y-axis signal is generated using an AWG of the LNHR DAC II.

<img src="./graphics/block-diagram-yaxis.png" width="1308" height="215">

### 5.1 - Configuring the AWG
The before chosen AWG-A is configured to the correct update rate. It must be checked, that the minimum AWG duration is at least 6 ms.

In [5]:
#calculate the internally used parameters
clock_period = int(acquisition_delay * 1000)
frequency = 1.0/(y_steps *(0.000001 * clock_period))
amplitude = y_stop_voltage - y_start_voltage
offset = y_start_voltage

#check minimum AWG duration
if(1.0 / frequency) < 0.006:
    raise Exception("Y-axis: clock period too short or not enough steps")

#configure AWG
dac.set_awg_channel(awg=f"{awg_output}", channel=f"{output_y}") #awg selected DAC-channel
dac.set_awg_cycles(awg=f"{awg_output}", cycles= 1)              #awg cycles set
dac.set_awg_trigger_mode(awg=f"{awg_output}", mode=0)           #awg external trigger
dac.set_awg_clock_period(board=f"{board}", period=clock_period) #clock period
dac.set_swg_adaptclock_state(state=0)                           #adaptive clock period

'0'

### 5.2 - standard waveform generation
The internal Standard Waveform Generator (SWG) is used to create the fast y-axis signal. It is configured to generate a ramp. It can also be used to generate simple standard waveforms for other applications.

In [6]:
dac.set_swg_mode(0)                             #swg mode (generate new/use old)
dac.set_swg_shape(shape=3)                            #set waveform
dac.set_swg_desired_frequency(frequency)        #set frequency
dac.set_swg_amplitude(amplitude)                #set amplitude
dac.set_swg_offset(offset)                      #set offset
dac.set_swg_phase(phase= 0.0000)                #set phase
dac.set_swg_wav_memory(memory)                  #set selected wave memory
dac.set_swg_selected_operation(operation=2)     #set selected wave memory operation

'0'

### 5.3 - write the generated waveform to the AWG
The before created waveform must be saved to the wave memory before it can be transferred to the AWG memory.

In [7]:
dac.clear_wav_memory(f"{awg_output}")                                           #clear wave memory
dac.apply_swg_operation()                                                       #apply swg to wave memory

last_mem_adr = dac.get_wav_memory_size(wav=f"{awg_output}")                     #get wave memory address
last_mem_adr = int(last_mem_adr)

dac.write(command=f"wav-{awg_output} {last_mem_adr:x} {y_start_voltage:.6f}")   #set last step to start value
dac.write_wav_to_awg(wav_awg=f"{awg_output}")                                   #write wave memory to awg memory



'0'

### 5.4 - setup the adaptive shift
A linear adaptive shift is executed when the parameter `adaptive_shift` is not zero. After each fast y-axis cycle, the start and stop voltage for the next cycle are shifted by the defined voltage. 

In [8]:
adaptive_scan = 1 if adaptive_shift > 0.0 else 0
dac.set_awg_start_mode(awg=f"{awg_output}", mode= 1)                    #normal/auto start awg
dac.set_awg_reload_mode(awg=f"{awg_output}", mode=adaptive_scan)        #keep/reload awg memory
dac.set_apply_polynomial(polynomial=f"{awg_output}", mode=adaptive_scan)#apply/skip polynomial and adaptive shift voltage

'0'

## 6 - setup trigger for data acquisition
The LNHR DAC II supports different trigger configurations.

<img src="./graphics/block-diagram-trigger.png" width="1308" height="215">

## 6.1 - using the trigger in-/outputs of the DAC

The DACs trigger can be setup to one of 4 configurations:

- No external trigger, scan as fast as possible
- Trigger from the DAQ to the DAC, the DAC is triggered by an external trigger, line by line
- Trigger from the DAC to the DAQ, the DAC generates a trigger signal, line by line
- Trigger from the DAC to the DAQ, the DAC generates a trigger signal, point by point

<img src="./graphics/trigger-configurations.png" width="585" height="332">

Each trigger configuration needs a slightly different hardware setup, using the logic in-/outputs on the backside of the DAC:

- Use the corresponding `TRIG IN AWG` output to trigger the DAC from an external trigger. Only line by line trigger is supported
- Use the corrresponding `SYNC OUT AWG` output to use the automatically generated trigger of the DAC. Only line by line trigger is supported
- see the following chapter if you want to use a point by point trigger

## 6.2 - using an AWG as a trigger output
In this example we are generating a point by point trigger which can be used to trigger the data acquisistion system. To do this, we are using a second of the DACs AWGs.

We generate a rectangular signal, which has double the update rate of the y-axis ramp. A caveat of this is, that we need different base clocks on the used AWGs. Since all the channels on a DAC-board (channels 1 to 12 and channels 13 to 24) are sharing a base clock, we need two AWGs on different DAC-boards. Therefore, this can only be done on a 24-channel LNHR DAC II.

To synchronize the generated rectangular signal to the y-axis signal, we simply need to connect the `SYNC OUT AWG` output, corresponding to the AWG that generates the y-axis signal, to the `TRIG IN AWG` input, corresponding to the AWG that generates the rectangular trigger signal. In this example we use the AWG-A to create the y-axis and the AWG-C to create the rectangular signal. This would mean, we need to connect the `SYNC OUT AWG-A` output to the `TRIG IN AWG-C`input of the DAC.

In [9]:
#define the DAC channel which will be used as output
output_daq_trigger = 13
#check availabilitiy of the awg-c and ramp-c, set auxiliary variables
if dac.read_AWGChannelAvailable(awg="C") == "1" and dac.read_rampChannelAvailable(ramp="C") == "1":
    awg_trigger = "c"
    memory = 2
else:
    raise Exception("AWG resources not available")

#configure AWG
dac.set_awg_channel(awg=f"{awg_trigger}", channel=f"{output_daq_trigger}") #awg selected dac-channel
dac.set_awg_cycles(awg=f"{awg_trigger}", cycles= y_steps)#awg cycles set
dac.set_awg_trigger_mode(awg=f"{awg_trigger}", mode= 1)#awg external trigger
dac.set_swg_adaptclock_state(state=1)#adaptive clock period

#create rectangular signal using the SWG
dac.set_swg_mode(mode= "0")                         #swg mode (generate new)
dac.set_swg_shape(shape=4)                          #set waveform
dac.set_swg_desired_frequency(1/(clock_period*0.000001))            #set frequency
dac.set_swg_amplitude(2.5)                    #set amplitude

dac.set_swg_offset(offset=2.500000)                 #swg offset
dac.set_swg_phase(phase=0.0000)                     #swg phase
dac.set_swg_dutycycle(dutycycle=50.000)             #swg duty cycle
dac.set_swg_selected_operation(operation= 2)        #swg selected wave memory operation

#write signal to wave memory and awg memory afterwards
dac.clear_wav_memory(wav=f"{awg_trigger}")          #clear wave memory
dac.apply_swg_operation()                           #apply swg to wave memory
dac.write_wav_to_awg(wav_awg=f"{awg_trigger}")      #write wave memory to awg

'0'

## 7 - prepare outputs and start 2D-scan
Before the 2D-scan can be started, the used outputs should be set to the assigned starting voltages. Additionally the Bandwidth should be set and the outputs should be turned on.

To start the 2D-scan, simply start the first cycle of the y-axis signal. If everything is configured correctly, the rest should happen fully automatic.


In [15]:
# set starting voltages
dac.set_chan_voltage(channel= output_x, dacvalue=int((float(x_start_voltage) + 10.000000) * 838860.74))
dac.set_chan_voltage(channel= output_y, dacvalue=int((float(y_start_voltage) + 10.000000) * 838860.74))

#set bandwith
dac.set_chan_bandwidth(channel=output_x, bandwidth="HBW")
dac.set_chan_bandwidth(channel=output_y, bandwidth="HBW")
dac.set_chan_bandwidth(channel=output_daq_trigger, bandwidth="HBW")

#turn on outputs
dac.set_chan_on(channel= output_x)
dac.set_chan_on(channel= output_y)
dac.set_chan_on(channel= output_daq_trigger)

#start 2D-scan
dac.set_awg_start_stop(awg=f"{awg_output}", command="start")


'0'

## 8 - restart or change parameters of the 2D-scan
It is recommended to turn all used outputs off and stop all AWGs and ramp generators before the 2D-scan is restarted or any parameters are changed.

In [14]:
if True:

    #turn off outputs
    dac.set_chan_off(channel=output_x)
    dac.set_chan_off(channel=output_y)
    dac.set_chan_off(channel=output_daq_trigger)

    #stop awgs and ramps

    dac.set_awg_start_stop(awg=f"{awg_output}", command="stop")
    dac.set_ramp_mode(ramp=f"{awg_output}", mode="stop")
    dac.set_awg_start_stop(awg=f"{awg_trigger}", command="stop")